In [51]:
with open("test_input_1.txt", "r") as f:
    test_input_1 = f.read()
with open("test_input_2.txt", "r") as f:
    test_input_2 = f.read()
with open("input.txt", "r") as f:
    real_input = f.read()
test_input_1


'LR\n\n11A = (11B, XXX)\n11B = (XXX, 11Z)\n11Z = (11B, XXX)\n22A = (22B, XXX)\n22B = (22C, 22C)\n22C = (22Z, 22Z)\n22Z = (22B, 22B)\nXXX = (XXX, XXX)'

In [52]:
def get_steps(raw_input):
    return raw_input.split("\n")[0]

get_steps(test_input_1)

'LR'

In [53]:
def get_route(raw_input):
    return raw_input.split("\n\n")[1].split("\n")

get_route(test_input_1)


['11A = (11B, XXX)',
 '11B = (XXX, 11Z)',
 '11Z = (11B, XXX)',
 '22A = (22B, XXX)',
 '22B = (22C, 22C)',
 '22C = (22Z, 22Z)',
 '22Z = (22B, 22B)',
 'XXX = (XXX, XXX)']

In [54]:
from collections import namedtuple
Node = namedtuple("Node", "l r")

In [55]:
def build_node(line):
    node, _, raw_left, raw_right = line.split()
    left = raw_left[1:-1]
    right = raw_right[:-1]
    return {node:Node(left, right)}

build_node(get_route(test_input_1)[0])

{'11A': Node(l='11B', r='XXX')}

In [56]:
def build_graph(lines):
    graph = dict()
    for line in lines:
        graph = graph | build_node(line)
    return graph

build_graph(get_route(test_input_1))

{'11A': Node(l='11B', r='XXX'),
 '11B': Node(l='XXX', r='11Z'),
 '11Z': Node(l='11B', r='XXX'),
 '22A': Node(l='22B', r='XXX'),
 '22B': Node(l='22C', r='22C'),
 '22C': Node(l='22Z', r='22Z'),
 '22Z': Node(l='22B', r='22B'),
 'XXX': Node(l='XXX', r='XXX')}

In [57]:
from itertools import cycle
START_NODE = "AAA"
END_NODE = "ZZZ"
def calculate_steps_to_end(graph, steps):
    current_node = graph[START_NODE]
    for i, step in enumerate(cycle(steps)):
        new_node_id = current_node.l if step == "L" else current_node.r

        if new_node_id == END_NODE:
            return i+1
        
        current_node = graph[new_node_id]

calculate_steps_to_end(build_graph(get_route(test_input_1)), get_steps(test_input_1))

KeyError: 'AAA'

In [ ]:
def get_result_1(raw_input):
    return calculate_steps_to_end(build_graph(get_route(raw_input)), get_steps(raw_input))

get_result_1(test_input_2)

6

In [ ]:
get_result_1(real_input)

19099

In [ ]:

def get_start_nodes(graph):
    return [id for id in graph.keys() if id[-1]=="A"]

get_start_nodes(build_graph(get_route(real_input)))

['HVA', 'LBA', 'FXA', 'GHA', 'PSA', 'AAA']

In [74]:
def get_advance_function(graph, steps):
    loop_cache = {}

    def get_steps_to_Z(start_node_id, start_step=0):
        nonlocal graph, steps, loop_cache
        step_start_index = start_step % len(steps)
        try:
            return start_step + loop_cache[(start_node_id, step_start_index)]
        except KeyError:
            pass

        current_node = graph[start_node_id]
        steps = steps[start_step:] + steps[:start_step]

        for i, step in enumerate(cycle(steps)):
            new_node_id = current_node.l if step == "L" else current_node.r

            if new_node_id[-1] == "Z":
                loop_cache[(new_node_id, step_start_index)] = i + 1
                return new_node_id, start_step + i + 1
            
            current_node = graph[new_node_id]
    
    return get_steps_to_Z

get_steps_to_Z("AAA", build_graph(get_route(real_input)), get_steps(real_input))

('ZZZ', 19099)

In [77]:
def get_convergent_steps(graph, steps):
    start_nodes = get_start_nodes(graph)
    advance_function = get_advance_function(graph, steps)
    step_counts = [advance_function(id) for id in start_nodes]

    while any(step_count != step_counts[0][1] for _, step_count in step_counts):
        to_advance = min(step_counts, key=lambda x: x[1])
        step_counts[step_counts.index(to_advance)] = advance_function(to_advance[0], start_step=to_advance[1])

    return step_counts[0][1]

def run(raw_input):
    return get_convergent_steps(build_graph(get_route(raw_input)), get_steps(raw_input))

run(test_input_1)
    
    


TypeError: cannot unpack non-iterable int object

In [66]:
run(real_input)

KeyboardInterrupt: 